In [1]:
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from binance_f import RequestClient
from binance_f.model import *
from binance_f.constant.test import *
from binance_f.base.printobject import *
import time
import threading

#### trading strategy reference: http://www.epchan.com/subscription/matlabexecution.pdf

# setup your api_key and api_secret, and connect to binance future

In [2]:

g_api_key = '' # binance api key
g_api_secret = '' # binance api secret

# connect to binance futures using api
request_client = RequestClient(api_key=g_api_key, secret_key=g_api_secret)

# function for return Bollinger's Band information

In [3]:
def get_bollinger_information(g_api_key, g_secret_key,symbol_):
    request_client = RequestClient(api_key=g_api_key, secret_key=g_secret_key)

    result = request_client.get_candlestick_data(symbol=symbol_, interval=CandlestickInterval.DAY1, 
                                                 startTime=None, endTime=None, limit=20)
    D = pd.DataFrame()
    D['open_date_time'] = [dt.datetime.fromtimestamp((result[x].closeTime / 1000)-86400) for x in range(len(result))]
    D['close'] = [result[x].close for x in range(len(result))]
    D['low'] = [result[x].low for x in range(len(result))]
    D['high'] = [result[x].high for x in range(len(result))]
    D['c_l_h_mean'] = (D['close'].astype(float)+D['low'].astype(float)+D['high'].astype(float))/3
    btc_df = D
    btc_df['std'] = btc_df['c_l_h_mean'].rolling(20).std(ddof=0)
    btc_df['MA'] = btc_df['c_l_h_mean'].rolling(20).mean()
    btc_df['BOLU'] = btc_df['MA'] + 2*btc_df['std']
    btc_df['BOLD'] = btc_df['MA'] - 2*btc_df['std']
    #btc_df['open_date_time'] = pd.to_datetime(btc_df['open_date_time'].dt.date)
    #btc_df['open_date_time'] = btc_df['open_date_time'].dt.strftime('%Y-%m-%d')

    btc_df = btc_df.dropna()
    return btc_df 

# function for our trading strategy

In [4]:
##
run_flag = 1
renew_frequency = 10

def trading_bot(symbol_ = "BTCUSDT", leverage_ = 2, contract_amount = 0.001, lookback_ = 20, entryZ_ = 2, exitZ_ = 1 ):
    
    global run_flag
    global renew_frequency
    
    now = dt.datetime.now()
    dt_string = now.strftime("%Y-%d-%m %H:%M:%S")
    symbol_to_trade_futures = symbol_ ## "BTCUSDT"
    ## accountName='U123456'

    leverage_ratio = leverage_#2
    numContract= contract_amount #0.001
    lookback= lookback_  # Number of periods used to compute Bollinger band ex.20
    entryZ= entryZ_ #2 # The distance between the upper and the lower band is 2*entryZ  ex. 2
    exitZ= exitZ_ # Exit when price revert to 1 standard deviation from the mean ex.1


    result = request_client.change_initial_leverage(symbol=symbol_to_trade_futures, leverage=leverage_ratio)# adjust the leverage ratio

    

    while run_flag == 1:

        # check position 
        ## pos = ............
        result_get_positions = request_client.get_position_v2()
        temp_num = 0
        for num, i in enumerate(result_get_positions):
            if i.symbol == symbol_to_trade_futures:
                temp_num = num
                break
        pos = result_get_positions[temp_num].positionAmt/numContract

        # check price for every 60 seconds
        ##time.sleep(60)
        time.sleep(renew_frequency)
        result = request_client.get_order_book(symbol = symbol_to_trade_futures, limit = 10)
        #get askPrice 
        askPrice = float(result.asks[0].price)
        #get bidPrice
        bidPrice = float(result.bids[0].price)

        #Calculate deviation of ask or bid price from moving average
        temp = get_bollinger_information(g_api_key , g_api_secret, symbol_to_trade_futures)
        mstd = temp['std'].to_list()[0]
        ma = temp['MA'].to_list()[0]
        zscoreAsk=(askPrice-ma)/mstd;
        zscoreBid=(bidPrice-ma)/mstd;

        print((ma+(2*mstd)),", ",ma,", ",(ma-(2*mstd)))

        if (pos == 0) and (zscoreAsk < (-entryZ)): #entry only
            #place order : market buy 1 numContract
            result = request_client.post_order(symbol=symbol_to_trade_futures, side=OrderSide.BUY, ordertype=OrderType.MARKET, 
                                               quantity=numContract, stopPrice=None, closePosition=False)
            PrintBasic.print_obj(result)
            #update pos : pos=numContract
        elif (pos < 0) and (zscoreAsk < (-entryZ)): #exit and entry
            #place order : market buy 2 numContract
            result = request_client.post_order(symbol=symbol_to_trade_futures, side=OrderSide.BUY, ordertype=OrderType.MARKET, 
                                               quantity= 2*numContract, stopPrice=None, closePosition=False)
            PrintBasic.print_obj(result)
            #update pos : pos=numContract
        elif (pos < 0) and (zscoreAsk < -exitZ): #exit only
            #place order : market buy 1 numContract
            result = request_client.post_order(symbol=symbol_to_trade_futures, side=OrderSide.BUY, ordertype=OrderType.MARKET, 
                                               quantity=numContract, stopPrice=None, closePosition=False)
            PrintBasic.print_obj(result)
            #update pos : pos=0
        elif (pos == 0) and (zscoreBid > entryZ): #entry only
            #place order : market sell 1 numContract
            result = request_client.post_order(symbol=symbol_to_trade_futures, side=OrderSide.SELL, ordertype=OrderType.MARKET, 
                                               quantity=numContract, stopPrice=None, closePosition=False)
            PrintBasic.print_obj(result)
            #update pos : pos=-numContract
        elif (pos > 0) and (zscoreBid > entryZ): #exit and entry
            #place order : market sell 2 numContract
            result = request_client.post_order(symbol=symbol_to_trade_futures, side=OrderSide.SELL, ordertype=OrderType.MARKET, 
                                               quantity= 2*numContract, stopPrice=None, closePosition=False)
            PrintBasic.print_obj(result)
            #update pos : pos=-numContract
        elif (pos > 0) and (zscoreBid > exitZ): #exit only
            #place order : market sell 1 numContract
            result = request_client.post_order(symbol=symbol_to_trade_futures, side=OrderSide.SELL, ordertype=OrderType.MARKET, 
                                               quantity=numContract, stopPrice=None, closePosition=False)
            PrintBasic.print_obj(result)
            #update pos : pos=0
    print('finish running')
    
def stop_robot():
    global run_flag
    global renew_frequency
    run_flag = 0
    time.sleep(renew_frequency*(1.2))
    run_flag = 1
    


# Run the robot

In [ ]:
t = threading.Thread(target = trading_bot, args=("BTCUSDT", 2, 0.001, 20, 2, 1 ))
t.start()

# Stop the robot

In [ ]:
stop_robot()